In [1]:
from IPython.display import clear_output
!pip install chart-studio
clear_output()
from sklearn.impute import KNNImputer
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from datetime import datetime as dt
import itertools
from tqdm import tqdm
from scipy.stats import norm, t
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
import gc
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn import metrics
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_columns', None)
plt.style.use('fivethirtyeight') 
from pylab import rcParams
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import statsmodels.api as sm
from numpy.random import normal, seed
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARIMA
import math
from sklearn.metrics import mean_squared_error
from catboost import Pool, CatBoostRegressor
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from keras.callbacks import EarlyStopping
import os 
import random
# import numpy as np 
from catboost import Pool, CatBoostRegressor

DEFAULT_RANDOM_SEED = 2021

def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
# tensorflow random seed 
import tensorflow as tf 
def seedTF(seed=DEFAULT_RANDOM_SEED):
    tf.random.set_seed(seed)
    
# torch random seed
import torch
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
      
# basic + tensorflow + torch 
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTF(seed)
    seedTorch(seed)
seedEverything(42)
seed = 42


In [2]:
# train

In [3]:
train_ori = pd.read_csv('/kaggle/input/airquoi/train.csv',parse_dates=['date'])
test_ori = pd.read_csv('/kaggle/input/airquoi/test.csv',parse_dates=['date'])
ss =pd.read_csv('/kaggle/input/airquoi/SampleSubmission.csv')

In [4]:
#

# Helper functions

In [5]:
def cust_metric(y_pred, ds):
    y_true = ds.get_label()
    sq_errs = abs(y_true - y_pred)
    avg_per_id = pd.Series(sq_errs).groupby(ds.site_code).mean()
#     print(avg_per_id)
    return 'avg_worst_by_id', avg_per_id.max(), False

def _get_X_Y_DF_from_CV(train_X, train_Y, train_index, validation_index):
        X_train, X_validation = (
            train_X.iloc[train_index],
            train_X.iloc[validation_index],
        )
        y_train, y_validation = (
            train_Y.iloc[train_index],
            train_Y.iloc[validation_index],
        )
        return X_train, X_validation, y_train, y_validation
def cust_score(y_predicted):
    l = []
    a = train.site_latitude.unique()
    for i in a:
        y_true = train.loc[train.site_latitude == i,'pm2_5']
        y_hat = y_predicted[y_true.index]
        l.append(metrics.mean_absolute_error(y_true,y_hat))
    return pd.DataFrame({'site_latitude':a,
                        'err': np.array(l)})
def preds_to_sub(test,preds,name = None,log = False):
    if log :
        test['pm2_5'] = np.exp(preds)
    else:
        test['pm2_5'] = preds
    sub = test[['ID','pm2_5']]
    if name:
        sub.to_csv(name+'.csv', index = False)
    return sub
def masked_w_aver(a,weights):
    ma = np.ma.MaskedArray(a, mask=np.isnan(a))
    return np.ma.average(ma, weights=weights)
def auto_regress(train,test,i,col_tar,output_col,preds_col):
    a = train.site_latitude.unique()
    time_ser = fancy_interpol(train,i,th=0.8,col_pred = 'pm2_5',col_tar = col_tar)
    time_ser = time_ser.resample('D').mean().interpolate()
    
    time_ser_test = test.loc[test.site_latitude == a[i]].sort_values(by = 'date').copy()
    time_ser_test.index = time_ser_test.date 
    time_ser_test = (time_ser_test[output_col])
    time_ser_test = time_ser_test.resample('D').mean().interpolate()

    train_sample = time_ser.values
    p_val = 0.05
    try:
        seas  = 1
        model = sm.tsa.SARIMAX(train_sample,order=(2,1,0),enforce_stationarity=True,
                       seasonal_order=(0,0,1,7),trend= None)
        result = model.fit(maxiter=1000,disp=False)
        assert result.pvalues[2]<p_val
    except:
        seas = 0
        model = sm.tsa.SARIMAX(train_sample,order=(2,1,0),trend=None)
        result = model.fit(maxiter=1000,disp=False)
#     print(result.summary())
#     predicted_result = result.predict(start=1, end=train_sample.shape[0]-1)
    y_true_dates = train.loc[train.site_latitude == a[i]].sort_values(by = 'date')['date'].values
    y_true = train.loc[train.site_latitude == a[i]].sort_values(by = 'date')['pm2_5'].values
    y_hat = train.loc[train.site_latitude == a[i]].sort_values(by = 'date')[preds_col].values
    correct = result.predict(start=1, end=time_ser.shape[0]+time_ser_test.shape[0])
    y_true_dates_test = test.loc[test.site_latitude == a[i]].sort_values(by = 'date')['date'].values
    mask_train = time_ser.index.isin(y_true_dates)
    mask_test = time_ser_test.index.isin(y_true_dates_test)
    train.loc[train.site_latitude == a[i],output_col] = correct[:time_ser.shape[0]][mask_train]
    test.loc[test.site_latitude == a[i],output_col] = correct[time_ser.shape[0]:][mask_test]
#     train.loc[train.site_latitude == a[i],'seas'] = seas
#     test.loc[test.site_latitude == a[i],'seas'] = seas
    return train, test
def fancy_interpol(train,i,th=0.8,col_pred = 'pm2_5',col_tar = 'pm2_5'):
#     th = 0.8
    a = train.site_latitude.unique()
    mi = train.pivot(index = 'date',columns = 'site_latitude',values = col_tar)[a[i]].dropna().index.min()
    ma = train.pivot(index = 'date',columns = 'site_latitude',values = col_tar)[a[i]].dropna().index.max()

    df_pred = train.pivot(index = 'date',columns = 'site_latitude',values = col_pred)[mi:ma]
    df_tar = train.pivot(index = 'date',columns = 'site_latitude',values = col_tar)[mi:ma]

    w_s = df_pred.corr()[a[i]].sort_values(ascending = False)
    filtered_ws = w_s[(w_s>th)& (w_s<1)]
    if len(filtered_ws)*len(df_tar[df_tar[a[i]].isna()])>0 : 
        df_tar.loc[df_tar[a[i]].isna(),a[i]] = df_tar.loc[df_tar[a[i]].isna(),filtered_ws.index].apply(lambda x: masked_w_aver(x, weights=filtered_ws.values),axis = 1)
    return df_tar.loc[mi:ma,a[i]].astype('float')

# Preprocessing

In [6]:
def preprocess(train,test,scale = True,target_encode = False,
               include_date = True,remove_nans = True, remove_multi = True):
    le = LabelEncoder()
    train['Train']=1
    test['Train']=0
    all_data=pd.concat([train,test])
    other_cols = ['pm2_5', 'Train', 'month', 'year', 'day',
               'week', 'dow', 'woy', 'rel_date']
    if include_date:
        all_data['month'] = all_data.date.dt.month
        all_data['year'] = all_data.date.dt.year
        all_data['day'] = all_data.date.dt.day
        all_data['week'] = all_data.date.dt.week
        all_data['dow'] = all_data.date.dt.dayofweek
        all_data['woy'] = all_data.date.dt.weekofyear
    all_data[['site_latitude','site_longitude']] =  all_data[['site_latitude','site_longitude']] .round(5)
    all_data['device'] = le.fit_transform(all_data['device'].values.reshape(-1,1))
    le = LabelEncoder()
    all_data['site_code'] = le.fit_transform(all_data.site_latitude.astype('str').values.reshape(-1,1))
    train = all_data.loc[all_data.Train == 1].reset_index(drop = True)
    test = all_data.loc[all_data.Train == 0].reset_index(drop = True)
    if target_encode:
        train,test = target_encode(train,test,target = 'pm2_5',groupss = 'site_code')
    if remove_nans:
        tr,te = cust_imputer(train.copy(), test.copy())
        all_data = pd.concat([tr,te]).reset_index(drop = True)
        s_codes = all_data.site_code.unique()
        distances = np.zeros((s_codes.shape[0],s_codes.shape[0]))
        for i in np.sort(s_codes):
            for j in np.sort(s_codes):
                distances[i,j] =  get_dist(all_data, i,j)
        distances = pd.DataFrame(index = np.sort(s_codes), columns = np.sort(s_codes),data = distances)
        SO2_cols = ['SulphurDioxide_SO2_column_number_density',
                    'SulphurDioxide_SO2_column_number_density_amf',
               'SulphurDioxide_SO2_slant_column_number_density',
               'SulphurDioxide_cloud_fraction',
                'SulphurDioxide_sensor_azimuth_angle',
               'SulphurDioxide_sensor_zenith_angle',
               'SulphurDioxide_solar_azimuth_angle',
               'SulphurDioxide_solar_zenith_angle',
               'SulphurDioxide_SO2_column_number_density_15km']
        CO_cols = ['CarbonMonoxide_CO_column_number_density',
               'CarbonMonoxide_H2O_column_number_density',
               'CarbonMonoxide_cloud_height', 'CarbonMonoxide_sensor_altitude',
               'CarbonMonoxide_sensor_azimuth_angle',
               'CarbonMonoxide_sensor_zenith_angle',
               'CarbonMonoxide_solar_azimuth_angle',
               'CarbonMonoxide_solar_zenith_angle']
        NO2_cols = ['NitrogenDioxide_NO2_column_number_density',
               'NitrogenDioxide_tropospheric_NO2_column_number_density',
               'NitrogenDioxide_stratospheric_NO2_column_number_density',
               'NitrogenDioxide_NO2_slant_column_number_density',
               'NitrogenDioxide_tropopause_pressure',
               'NitrogenDioxide_absorbing_aerosol_index',
               'NitrogenDioxide_cloud_fraction', 'NitrogenDioxide_sensor_altitude',
               'NitrogenDioxide_sensor_azimuth_angle',
               'NitrogenDioxide_sensor_zenith_angle',
               'NitrogenDioxide_solar_azimuth_angle',
               'NitrogenDioxide_solar_zenith_angle']
        HCHO_cols = ['Formaldehyde_tropospheric_HCHO_column_number_density',
               'Formaldehyde_tropospheric_HCHO_column_number_density_amf',
               'Formaldehyde_HCHO_slant_column_number_density',
               'Formaldehyde_cloud_fraction', 'Formaldehyde_solar_zenith_angle',
               'Formaldehyde_solar_azimuth_angle', 'Formaldehyde_sensor_zenith_angle',
               'Formaldehyde_sensor_azimuth_angle']
        UV_cols = ['UvAerosolIndex_absorbing_aerosol_index',
               'UvAerosolIndex_sensor_altitude', 'UvAerosolIndex_sensor_azimuth_angle',
               'UvAerosolIndex_sensor_zenith_angle',
               'UvAerosolIndex_solar_azimuth_angle',
               'UvAerosolIndex_solar_zenith_angle']
        O3_cols = ['Ozone_O3_column_number_density_amf',
               'Ozone_O3_slant_column_number_density',
               'Ozone_O3_effective_temperature', 'Ozone_cloud_fraction',
               'Ozone_sensor_azimuth_angle', 'Ozone_sensor_zenith_angle',
               'Ozone_solar_azimuth_angle', 'Ozone_solar_zenith_angle']
        cloud_cols = ['Cloud_cloud_fraction', 'Cloud_cloud_top_pressure',
               'Cloud_cloud_top_height', 'Cloud_cloud_base_pressure',
               'Cloud_cloud_base_height', 'Cloud_cloud_optical_depth',
               'Cloud_surface_albedo', 'Cloud_sensor_azimuth_angle',
               'Cloud_sensor_zenith_angle', 'Cloud_solar_azimuth_angle',
               'Cloud_solar_zenith_angle']
        target = ['pm2_5']
        all_cols = SO2_cols+cloud_cols+O3_cols+UV_cols+HCHO_cols+NO2_cols+CO_cols

        other_cols_test = ['Train', 'month', 'year', 'day',
               'week', 'dow', 'woy', 'rel_date']
        for col in tqdm(all_cols):
            for s_co in s_codes:
                all_data = distance_w_impute(col,s_co,all_data,distances)
        test = all_data[all_data.ID.isin(test_ori.ID)].drop_duplicates().reset_index(drop = True)
        train = all_data[all_data.ID.isin(train_ori.ID)].drop_duplicates().reset_index(drop = True)
        kn = KNNImputer(n_neighbors=5)
        cols = train.drop('pm2_5',axis = 1).select_dtypes(include = ['float']).columns
        train[cols] = kn.fit_transform(train[cols])
        test[cols] = kn.transform(test[cols])
#         cols = 
        if scale:
            rs= RobustScaler()
            train[cols] = rs.fit_transform(train[cols])
            test[cols] = rs.transform(test[cols])
    if remove_multi:
        cols = train.select_dtypes(include = ['int','float']).columns
        l_drop = []
        for col in cols:
            df = train.pivot(index = 'date',columns = 'site_code',values = col)
            if df.corr().mean().mean()>0.99:
                l_drop.append(col)
        train = train.drop([l for l  in l_drop[1:] if l not in other_cols],axis = 1 )
        test = test.drop([l for l  in l_drop[1:] if l not in other_cols],axis = 1 )
    return train.sort_index(), test.sort_index()
def get_dist(all_data, i,j):
    return np.sqrt((all_data.loc[all_data.site_code == i,'site_latitude'].unique()[0]- all_data.loc[all_data.site_code == j,'site_latitude'].unique()[0])**2 + (all_data.loc[all_data.site_code == i,'site_longitude'].unique()[0]- all_data.loc[all_data.site_code == j,'site_longitude'].unique()[0])**2)

def distance_w_impute(col,s_co,all_data,distances):
    ind = all_data.loc[(all_data.site_code == s_co)& (all_data[col].isna()),'date'].values
    df3 = all_data.loc[(all_data.site_code != s_co)& (all_data.date.isin(ind)),['date',col,'site_code']]
    df3['weights'] = df3['site_code'].map(distances[s_co]**(-1))
    df3[col] = df3[col]*df3['weights']
    maps = df3.groupby('date').apply(lambda x : x[col].sum()/(x['weights'].sum()))
    all_data.loc[(all_data.site_code == s_co)& (all_data[col].isna()),col] = all_data.loc[(all_data.site_code == s_co)& (all_data[col].isna()),'date'].map(maps)
    return all_data
def cust_imputer(train, test):
    all_data = pd.concat([train,test]).reset_index(drop = True)
    sensor_az_ang = ['SulphurDioxide_sensor_azimuth_angle','CarbonMonoxide_sensor_azimuth_angle','NitrogenDioxide_sensor_azimuth_angle',
          'Formaldehyde_sensor_azimuth_angle','UvAerosolIndex_sensor_azimuth_angle','Ozone_sensor_azimuth_angle',
         'Cloud_sensor_azimuth_angle']
    sensor_zen_ang = ['SulphurDioxide_sensor_zenith_angle','CarbonMonoxide_sensor_zenith_angle','NitrogenDioxide_sensor_zenith_angle',
          'Formaldehyde_sensor_zenith_angle','UvAerosolIndex_sensor_zenith_angle','Ozone_sensor_zenith_angle',
         'Cloud_sensor_zenith_angle']
    solar_az_ang = ['SulphurDioxide_solar_azimuth_angle','CarbonMonoxide_solar_azimuth_angle','NitrogenDioxide_solar_azimuth_angle',
          'Formaldehyde_solar_azimuth_angle','UvAerosolIndex_solar_azimuth_angle','Ozone_solar_azimuth_angle',
         'Cloud_solar_azimuth_angle']
    solar_zen_ang = ['SulphurDioxide_solar_zenith_angle','CarbonMonoxide_solar_zenith_angle','NitrogenDioxide_solar_zenith_angle',
          'Formaldehyde_solar_zenith_angle','UvAerosolIndex_solar_zenith_angle','Ozone_solar_zenith_angle',
         'Cloud_solar_zenith_angle']
    X =  all_data[sensor_az_ang]
    imputer = KNNImputer(n_neighbors=5)
    all_data.loc[X.isna().sum(axis = 1)<7,sensor_az_ang] = imputer.fit_transform(X[X.isna().sum(axis = 1)<7]) 
    X =  all_data[sensor_zen_ang]
    imputer = KNNImputer(n_neighbors=5)
    all_data.loc[X.isna().sum(axis = 1)<7,sensor_zen_ang] = imputer.fit_transform(X[X.isna().sum(axis = 1)<7])
    X =  all_data[solar_az_ang]
    imputer = KNNImputer(n_neighbors=5)
    all_data.loc[X.isna().sum(axis = 1)<7,solar_az_ang] = imputer.fit_transform(X[X.isna().sum(axis = 1)<7]) 
    X =  all_data[solar_zen_ang]
    imputer = KNNImputer(n_neighbors=5)
    all_data.loc[X.isna().sum(axis = 1)<7,solar_zen_ang] = imputer.fit_transform(X[X.isna().sum(axis = 1)<7])
    return all_data.loc[:train.shape[0]], all_data.loc[train.shape[0]:]


# Models

In [7]:
def train_keras(baseline_model,train,features,target,test_data,kf,split_by):
    inp = train[features].shape[1]
    cats = train[split_by].values
    X_scaled = train[features].values
    Y = train[target].values
    
    y_oof = np.zeros(shape=(len(X_scaled),1))
    y_predicted = np.zeros(shape=(len(test_data),1))
    cv_scores = []
    test_data = test_data[features]
    models = []
    for i, (train_index, test_index) in enumerate(kf.split(X_scaled, cats)):
        print(' keras kfold: {}  of  {} : '.format(i+1, K_FOLDS ))
        X_train, X_valid = X_scaled[train_index], X_scaled[test_index]
        y_train, y_valid = Y[train_index], Y[test_index]
        my_model = baseline_model(inp)
        my_model.fit(X_train, y_train,
                     validation_data=(X_valid, y_valid),
                     epochs=100,
                     batch_size = 64,
                     callbacks=[EarlyStopping(patience=20)],
                     verbose=0)
        
        models.append(my_model)
        y_oof[test_index] = my_model.predict(X_valid)
        y_predicted += my_model.predict(test_data.values) 
        del my_model
        gc.collect()
        cv_oof_score = metrics.mean_absolute_error(y_valid, y_oof[test_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {i+1} is {cv_oof_score}")

#         del validation_index, X_validation, y_validation
#         gc.collect()

    y_predicted /= K_FOLDS
    oof_score = round(metrics.mean_absolute_error(Y, y_oof), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted,models,y_oof,oof_score 
def train_lgb(X,features,target,test_data,params,kf,split_by):
    features_importance= pd.DataFrame({'Feature':[], 'Importance':[]})
    models =[]
    train_X = X[features]
    train_Y = X[target]
    split_by = X[split_by]
    test_data = test_data[features]
    test_X = test_data.copy()
    print(f"Shape of train_X : {train_X.shape}, test_X: {test_X.shape}, train_Y: {train_Y.shape}")
    
    predictors = list(train_X.columns)
    # print(f"List of features to be used {list(predictors)}")

    # Selecting n_splits to be 3, since class 42 has 
    # just 3 instances
#     kf = KFold(random_state=seed_lgb,n_splits=K_FOLDS, shuffle=shuffle_lgb)
#     kf = StratifiedKFold(random_state=seed_lgb,n_splits=K_FOLDS, shuffle=shuffle_lgb)
    y_oof_lgb = np.zeros(shape=(len(train_X),))
    y_predicted_lgb = np.zeros(shape=(len(test_X),))
    cv_scores = []
    fold = 0
    n_folds = kf.get_n_splits()
    for train_index, validation_index in kf.split(X=train_X, y=split_by):
        fold += 1
        print(f"fold {fold} of {n_folds}")

        X_train, X_validation, y_train, y_validation = _get_X_Y_DF_from_CV(
            train_X, train_Y, train_index, validation_index
        )

        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_validation, y_validation, reference=lgb_train)

        model = lgb.train(
            params,
            lgb_train,
            valid_sets=[lgb_train, lgb_eval],
            verbose_eval=-1,
#             early_stopping_rounds=EARLY_STOPPING_ROUNDS,
#             num_boost_round=N_ESTIMATORS,
            feature_name=predictors,
            categorical_feature="auto",
        )
        del lgb_train, lgb_eval, train_index, X_train, y_train
        gc.collect()

        y_oof_lgb[validation_index] = model.predict(
            X_validation, num_iteration=model.best_iteration
        )

        y_predicted_lgb += model.predict(
            test_data.values, num_iteration=model.best_iteration
        )
        fold_importance_df= pd.DataFrame({'Feature':[], 'Importance':[]})
        fold_importance_df['Feature']= predictors
        fold_importance_df['Importance']= model.feature_importance()
        fold_importance_df["fold"] = fold + 1
        features_importance = pd.concat([features_importance, fold_importance_df], axis=0)
        models.append(model)

        best_iteration = model.best_iteration
        print(f"Best number of iterations for fold {fold} is: {best_iteration}")

        cv_oof_score = metrics.mean_absolute_error(y_validation, y_oof_lgb[validation_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {fold} is {cv_oof_score}")

        del validation_index, X_validation, y_validation
        gc.collect()

    y_predicted_lgb /= n_folds
    oof_score = round(metrics.mean_absolute_error(train_Y, y_oof_lgb), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted_lgb,models,y_oof_lgb,oof_score,features_importance
def train_catbo(train_X,features, target,test_X,params,kf,split_by):
    y_oof = np.zeros(shape=(len(train_X),))
    y_predicted = np.zeros(shape=(len(test_X),))
    train_Y= train_X[target]
    split_by = train_X[split_by]
    train_X = train_X[features]
    test_X = test_X[features]
    cv_scores = []
    models = []
    fold = 0
    n_folds = kf.get_n_splits()
    for train_index, validation_index in kf.split(X=train_X, y=split_by):
        fold += 1
        print(f"fold {fold} of {n_folds}")
        X_train, X_validation, y_train, y_validation = _get_X_Y_DF_from_CV(
            train_X, train_Y, train_index, validation_index
        )
        train_pool = Pool(data=X_train, label=y_train)
        eval_pool = Pool(data=X_validation, label=y_validation.values) 
        model = CatBoostRegressor(**params)
        model.fit(train_pool,plot=True,eval_set=eval_pool)
        del train_index, X_train, y_train
        gc.collect()
        models.append(model)
        y_oof[validation_index] = model.predict(
            X_validation )

        y_predicted += model.predict(
            test_X.values
        )
        cv_oof_score = metrics.mean_absolute_error(y_validation, y_oof[validation_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {fold} is {cv_oof_score}")

        del validation_index, X_validation, y_validation
        gc.collect()

    y_predicted /= n_folds
    oof_score = round(metrics.mean_absolute_error(train_Y, y_oof), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted,models,y_oof,oof_score
def train_xgb(X,features,target,test_data,params,kf,split_by,num_iter=1500,es = 100,ve = 0):
#     features = X.columns
    y = X[target]
    split_by = X[split_by]
    X = X[features].values
    y_oof = np.zeros(shape=(len(X),))
    y_predicted = np.zeros(shape=(len(test_data), ))
    test_data = test_data[features]
    cv_scores = []
    models = []
    n_folds = kf.get_n_splits()
    for i, (train_index, test_index) in enumerate(kf.split(X, split_by)):
        print(' xgb kfold: {}  of  {} : '.format(i+1, n_folds ))
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]
        d_train = xgb.DMatrix(X_train, y_train) 
        d_valid = xgb.DMatrix(X_valid, y_valid) 
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        xgb_model = xgb.train(params, d_train, num_iter, watchlist,
                              early_stopping_rounds=es, 
                            verbose_eval=ve)
        models.append(xgb_model)
        y_oof[test_index] = xgb_model.predict(xgb.DMatrix(X_valid), 
                            ntree_limit=xgb_model.best_ntree_limit)
        y_predicted += xgb_model.predict(xgb.DMatrix(test_data[features].values), 
                            ntree_limit=xgb_model.best_ntree_limit) 
        
        cv_oof_score = metrics.mean_absolute_error(y_valid, y_oof[test_index])
        cv_scores.append(cv_oof_score)
        print(f"CV OOF Score for fold {i+1} is {cv_oof_score}")

    y_predicted /= n_folds
    oof_score = round(metrics.mean_absolute_error(y, y_oof), 5)
    avg_cv_scores = round(sum(cv_scores) / len(cv_scores), 5)
    std_cv_scores = round(np.array(cv_scores).std(), 5)
    return y_predicted,models,y_oof,oof_score 
def train_aaa(train,test):
    tr,te = train_all(train.copy(),test.copy(),target,features,SEED,K_fols,seeds,shuffles)
    train_99 = train[train.pm2_5<train.pm2_5.quantile(0.999)].reset_index(drop = True).copy()
    tr_99,te_99 = train_all(train_99,test.copy(),target,features,SEED,K_fols,seeds,shuffles)
    train_999 = train[train.pm2_5<train.pm2_5.quantile(0.999)].reset_index(drop = True).copy()
    tr_999,te_999 = train_all(train_999,test.copy(),target,features,SEED,K_fols,seeds,shuffles)
    cols = ['ID','pm2_5', 'month', 'year', 'day',
       'week', 'dow', 'woy', 'site_code', 'preds_lgb', 'preds_xgb',
       'preds_cat']
    tr_keras = tr[['month', 'year', 'day',
       'week', 'dow', 'woy', 'site_code', 'preds_lgb', 'preds_xgb',
       'preds_cat','pm2_5']]
    a_train, a_test, b_train, b_test = train_test_split(tr_keras.drop(target,axis = 1), 
                                                    tr_keras[target], test_size=0.5,
                                                    random_state=20)
    tas,vel = keras_overfit(a_train,b_train,a_test,b_test,te[a_train.columns])
    tr['preds0'] = vel
    te['preds0'] = tas
    tr_keras = tr_99[['month', 'year', 'day',
       'week', 'dow', 'woy', 'site_code', 'preds_lgb', 'preds_xgb',
       'preds_cat','pm2_5']]
    a_train, a_test, b_train, b_test = train_test_split(tr_keras.drop(target,axis = 1), 
                                                    tr_keras[target], test_size=0.5,
                                                    random_state=20)
    tas,vel = keras_overfit(a_train,b_train,a_test,b_test,te_99[a_train.columns])
    tr_99['preds1'] = vel
    te_99['preds1'] = tas
    tr_keras = tr_999[['month', 'year', 'day',
       'week', 'dow', 'woy', 'site_code', 'preds_lgb', 'preds_xgb',
       'preds_cat','pm2_5']]
    a_train, a_test, b_train, b_test = train_test_split(tr_keras.drop(target,axis = 1), 
                                                    tr_keras[target], test_size=0.5,
                                                    random_state=20)
    tas,vel = keras_overfit(a_train,b_train,a_test,b_test,te_999[a_train.columns])
    tr_999['preds2'] = vel
    te_999['preds2'] = tas
    tr.index = tr.ID
    te.index = te.ID
    tr_99.index = tr_99.ID
    te_99.index = te_99.ID
    tr_999.index = tr_999.ID
    te_999.index = te_999.ID
    train['preds0'] = train['ID'].map(tr['preds0'])
    test['preds0'] = test['ID'].map(te['preds0'])
    train['preds1'] = train['ID'].map(tr_99['preds1'])
    test['preds1'] = test['ID'].map(te_99['preds1'])
    train['preds2'] = train['ID'].map(tr_999['preds2'])
    test['preds2'] = test['ID'].map(te_999['preds2'])
    train['errors0'] = train['pm2_5']-train['preds0']
    train['errors1'] = train['pm2_5']-train['preds1']
    train['errors2'] = train['pm2_5']-train['preds2']
    a = train.site_latitude.unique()
    for j in range(3):
        train['correct'+str(j)]= 0
        test['correct'+str(j)] = 0
        for i in range(a.shape[0]):
            train,test = auto_regress(train,test,i,'errors'+str(j),'correct'+str(j),'preds'+str(j))
        train['refined_preds'+str(j)] = train['preds'+str(j)] + train['correct'+str(j)]
    return train, test

def train_all(train,test,target,features,SEED,k_folds,seeds,shuffles):
    seedEverything(SEED)
    
    kf_cat = KFold(n_splits = K_fols['cat'], shuffle=shuffles['cat'],
                   random_state = seeds['cat'])
    y_predicted_cat,models_cat,y_oof_cat,oof_score_cat =train_catbo(train,features,target,test,params['params_cat'],
                                                                    kf_cat, 'pm2_5')
    seedEverything(SEED)

    kf_xgb = KFold(n_splits = K_fols['xgb'], shuffle=shuffles['xgb'],
                   random_state = seeds['xgb'])

    y_predicted_xgb,models_xgb,y_oof_xgb,oof_score_xgb=train_xgb(train,features,target,test,
                                                             params['params_xgb'],kf_xgb,'pm2_5',num_iter=1000,es = 30,ve = 100)
    
    seedEverything(SEED)
    kf_lgb = KFold(n_splits = K_fols['lgb'], shuffle=shuffles['lgb'],random_state = seeds['lgb'])

    y_predicted_lgb,models_lgb,y_oof_lgb,oof_score_lgb,features_importance_lgb = train_lgb(train,features,target,test,params['params_lgb'],kf_lgb,'pm2_5')

    train['preds_lgb'] = y_oof_lgb                                                                                 
    train['preds_xgb'] = y_oof_xgb
    train['preds_cat'] = y_oof_cat                                                                                 
    test['preds_lgb'] = y_predicted_lgb                                                                                 
    test['preds_xgb'] = y_predicted_xgb
    test['preds_cat'] = y_predicted_cat                                                                                 
    return train,test


# Parameters and features

In [9]:
SEED = 42
seeds = {'lgb':None,
        'cat': 42,
        'xgb':43}
shuffles ={'lgb' : False,
          'cat' : True,
          'xgb' : True}
K_fols = {'lgb':3,
         'cat':4,
         'xgb':4}
lgb_params  = {'objective': 'regression',
            'metric': 'mae',
            'n_estimators':5000,
            'verbose':-1,
    'learning_rate': 0.0635938724410916,
 'num_leaves': 31,
 'colsample_bytree': 0.2556455958822984,
 'subsample': 0.29787449702328844,
 'max_depth': 10,
 'min_child_samples': 16,
 'reg_alpha': 1.4868037975722531e-06,
 'reg_lambda': 0.0005428092641970505,
 'cat_smooth': 95} 
params_cat =  {'eval_metric': 'RMSE',
#                              'iterations': 10,
                'verbose' : 0,
                'use_best_model': True,
                'random_seed' : seeds['cat'],
                'learning_rate': 0.07574755563809993, 'depth': 6, 'l2_leaf_reg': 0.39323612155779863, 'random_strength': 0.20259568870497338, 'grow_policy': 'Lossguide', 'max_bin': 67, 'min_data_in_leaf': 10, 'bootstrap_type': 'Bernoulli', 'subsample': 0.9244602042319705}
params_xgb = {'objective': 'reg:squarederror',
              'eval_metric': 'mae','grow_policy': 'lossguide', 'learning_rate': 0.03982304720613181, 'max_depth': 12, 'max_delta_step': 10, 'min_child_weight': 133, 'colsample_bytree': 0.8709673399493595, 'subsample': 0.7129355859703879, 'max_leaves': 34}
   
params={'params_lgb' : lgb_params,
      'params_xgb': params_xgb,
      'params_cat':params_cat}
drop_cols = ['ID','date','Train','pm2_5','site_latitudepm2_5enc7',
       'site_latitudepm2_5enc8', 'site_latitudepm2_5enc6',
       'site_latitudepm2_5enc5', 'site_latitudepm2_5enc4',
       'site_latitudepm2_5enc', 'site_latitudepm2_5enc1',
       'site_latitudepm2_5enc2', 'site_latitudepm2_5enc3','preds']
target = 'pm2_5'

In [ ]:
train,test = preprocess(train_ori,test_ori,scale = True,target_encode = False,
              include_date = True,remove_nans = False, remove_multi = False)
features  = [d for d in test.columns if d  not in drop_cols]

tr_1,te_1 = train_aaa(train,test)
train,test = preprocess(train_ori,test_ori,scale = True,target_encode = False,
              include_date = True,remove_nans = True, remove_multi = True)
features  = [d for d in test.columns if d  not in drop_cols]

tr_2,te_2 = train_aaa(train,test)
train,test = preprocess(train_ori,test_ori,scale = True,target_encode = True,
              include_date = True,remove_nans = True, remove_multi = False)
features  = [d for d in test.columns if d  not in drop_cols]

tr_3,te_3 = train_aaa(train,test)
train,test = preprocess(train_ori,test_ori,scale = True,target_encode = True,
              include_date = True,remove_nans = True, remove_multi = True)
features  = [d for d in test.columns if d  not in drop_cols]

tr_4,te_4 = train_aaa(train,test)


fold 1 of 4


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CV OOF Score for fold 1 is 7.067223042355893
fold 2 of 4


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CV OOF Score for fold 2 is 7.216686751580745
fold 3 of 4


...

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CV OOF Score for fold 4 is 7.696428317528332
 xgb kfold: 1  of  4 : 
[0]	train-mae:56.06458	valid-mae:56.64251
[100]	train-mae:22.52397	valid-mae:23.31667
[200]	train-mae:11.75842	valid-mae:12.83409
[300]	train-mae:8.78974	valid-mae:10.26200
[400]	train-mae:7.50924	valid-mae:9.34085
[500]	train-mae:6.80118	valid-mae:8.94031
[600]	train-mae:6.26975	valid-mae:8.68247
[700]	train-mae:5.85660	valid-mae:8.55866
[800]	train-mae:5.51960	valid-mae:8.46620
[900]	train-mae:5.22919	valid-mae:8.38877
[999]	train-mae:4.98517	valid-mae:8.32844
CV OOF Score for fold 1 is 8.327862326797186
 xgb kfold: 2  of  4 : 
[0]	train-mae:56.21452	valid-mae:56.19273
[100]	train-mae:22.73260	valid-mae:22.65811
[200]	train-mae:11.93064	valid-mae:12.35795
[300]	train-mae:9.03697	valid-mae:9.91649
[400]	train-mae:7.73298	valid-mae:9.05789
[500]	train-mae:7.00870	valid-mae:8.64184
[600]	train-mae:6.48426	valid-mae:8.41069
[700]	train-mae:6.08165	valid-mae:8.28360
[800]	train-mae:5.75449	valid-mae:8.20164
[900]	train-m

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CV OOF Score for fold 1 is 6.976930397515213
fold 2 of 4


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
# test

In [ ]:
# tr_99.columns
# tr_99.columns

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
# tas.shape

In [ ]:
# tas,vel = keras_overfit(a_train,b_train,a_test,b_test,te_99[a_train.columns])

In [ ]:
def keras_overfit(_train,_train_y,_val,_val_y,test):
    seedEverything(SEED)
    all_d = pd.concat([_train,_val]).sort_index()
    inp = _train.shape[1]
    y_oof = np.zeros(all_d.shape[0])
    model = Sequential()
    model.add(Dense(1, input_dim = inp, activation = 'linear')) # Rectified Linear Unit Activation Function
    model.compile(loss = 'mse', optimizer = tf.keras.optimizers.RMSprop(lr=0.01),
                  metrics = ["mean_absolute_error"])
    model.fit(_train, _train_y,
                     validation_data=(_val, _val_y),
                     epochs=5,
                     batch_size = 64,
                     callbacks=[EarlyStopping(patience=20)],
                     verbose=1)
    y_oof = model.predict(all_d)
    y_predicted = model.predict(test.values)
    return y_predicted,y_oof

In [ ]:
# tr.groupby('site_code').apply(lambda x : metrics.mean_absolute_error(x['pm2_5'],x['preds_xgb'])).sort_values(ascending = False)
# te_99['pm2_5'] = te_99[['preds_lgb','preds_cat','preds_xgb']].mean(axis = 1)
# tr_keras = tr[['']]



In [ ]:
# te_999[['ID','pm2_5']].to_csv('goofy2.csv', index = False)

In [ ]:
from keras import optimizers
def baseline_model(inp):
    # Create model here
    model = Sequential()
    model.add(Dense(100, input_dim = inp, activation = 'tanh')) # Rectified Linear Unit Activation Function
    model.add(Dropout(0.3))

#     model.add(Dense(32, activation = 'tanh'))
    model.add(Dense(8, activation = 'linear'))
    
    model.add(Dense(1, activation = 'linear')) # Softmax for multi-class classification
    # Compile model here
    model.compile(loss = 'mse', optimizer = tf.keras.optimizers.RMSprop(lr=0.025),
                  metrics = ['mse'],)
    return model

In [ ]:
 
SEED = 42
seedEverything(SEED)
K_FOLDS = 2
kf = KFold(n_splits = K_FOLDS, shuffle=False,random_state = None)
split_by = 'pm2_5'
y_predicted_keras,models_keras,y_oof_keras,oof_score_keras = train_keras(bl,train,features,train['pm2_5'],test,kf,split_by)

In [ ]:
preds_blend,_,w = models_to_blend()
# w

In [ ]:
preds_to_sub(test,preds_blend,'blend31')

In [ ]:
EARLY_STOPPING_ROUNDS = 30
N_ESTIMATORS = 5000
K_FOLDS = 2
lgb_params = {
    "objective": "regression",
    "boosting_type": "gbdt",
    "learning_rate": 0.1,
    "n_jobs": 4,
    "seed": SEED,
    "max_depth": 13,
#     "num_leaves":10,
    "force_col_wise":True,
#     'min_data_in_leaf' : 5,
#     'feature_fraction':0.5,
#     'bagging_fraction':0.8,
#     "max_bin": 255,
#     'reg_lambda': 0.1,  # L1 regularization term on weights
#     'reg_lambda': 10,
    "metric": "MAE",
    "verbose": -1,
}
kf = KFold(n_splits = K_FOLDS, shuffle=False,random_state = None)
y_predicted_lgb,models_lgb,y_oof_lgb,oof_score_lgb,features_importance_lgb = train_lgb(train,features,
                                                                                       target,test,
                                                                                       lgb_params,kf,'pm2_5')

In [ ]:
target

In [ ]:
from catboost import Pool, CatBoostRegressor
params_cat = {'iterations':5000,
        'learning_rate':0.025,
#         'random_strength':0.1,
        'early_stopping_rounds':30, 
        'depth':3,
        'loss_function':'RMSE',
        'eval_metric':'MAE',
        'verbose' : 100,
#         'leaf_estimation_method':'Newton'
             }
kf = KFold(n_splits = K_FOLDS, shuffle=True,random_state = SEED)

y_predicted_cat,models_cat,y_oof_cat,oof_score_cat =train_catbo(train[features],
                                                                        train['pm2_5'],test[features],params_cat,
                                                                kf, 'pm2_5')

In [ ]:
params_xgb = {"objective":"reg:squarederror",'learning_rate': 0.1,
           'max_depth': 3}#, 'subsample': 0.9,
#           'colsample_bytree': 0.9}

y_predicted_xgb,models_xgb,y_oof_xgb,oof_score_xgb=train_xgb(train[features],
                                                                        train['pm2_5'],test[features],
                                                             params_xgb,kf,'pm2_5',num_iter=1000,es = 30,ve = 100)

In [ ]:
def models_to_blend():
    w_cat = cust_score(y_oof_cat).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()
    w_lgb = cust_score(y_oof_lgb).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()
    w_xgb = cust_score(y_oof_xgb).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()
    w_keras = cust_score(y_oof_keras).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()
    weigths = [1/w_cat,1/w_lgb,1/w_xgb,1/w_keras]
    blend_preds = np.average(np.vstack([y_predicted_cat,y_predicted_xgb,y_predicted_cat,y_predicted_keras.squeeze()]),axis = 0,weights=weigths)
    blend_preds_train = np.average(np.vstack([y_oof_cat,y_oof_xgb,y_oof_cat,y_oof_keras.squeeze()]),axis = 0,weights=weigths)
    w_blend = cust_score(blend_preds_train).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()
    weights = [w_cat,w_lgb,w_xgb,w_keras,w_blend]
    print("mae of cat : {}.-------- worst {}".format(metrics.mean_absolute_error(train['pm2_5'],y_oof_cat).round(3),w_cat.round(3)))
    print("mae of lgb : {}..-------- worst {}".format(metrics.mean_absolute_error(train['pm2_5'],y_oof_lgb).round(3),w_lgb.round(3)))
    print("mae of xgb : {}..-------- worst {}".format(metrics.mean_absolute_error(train['pm2_5'],y_oof_xgb).round(3),w_xgb.round(3)))
    print("mae of keras : {}..-------- worst {}".format(metrics.mean_absolute_error(train['pm2_5'],y_oof_keras).round(3),w_keras.round(3)))
    print("mae of blend : {}..-------- worst {}".format(metrics.mean_absolute_error(train['pm2_5'],blend_preds_train).round(3),w_blend.round(3)))
          
    return blend_preds,blend_preds_train,weights
    

In [ ]:
_,w = models_to_blend()

In [ ]:
w

In [ ]:
np.vstack([y_predicted_lgb,y_predicted_xgb]).shape

In [ ]:
# y_predicted_xgb

In [ ]:
np.average(np.vstack([y_predicted_lgb,y_predicted_xgb]),axis = 0,weights=[10,1])

In [ ]:
preds_to_sub(test,y_predicted_lgb+,'lgb_scaled3')

In [ ]:
import optuna
SEED = 42
K_FOLDS = 4
target = 'pm2_5'
def objective(trial):
    params = {
                #'iterations' : 10000, replaced by early stopping
                'eval_metric': 'RMSE',
                'verbose' : 500,
                'use_best_model': True,
                'random_seed' : SEED,
                'learning_rate' :trial.suggest_loguniform('learning_rate', 0.01, 0.3),
                "depth": trial.suggest_int("depth", 1, 9),
                'l2_leaf_reg' :trial.suggest_loguniform('l2_leaf_reg', 1e-8, 10),
                'random_strength' : trial.suggest_loguniform('random_strength', 0.1, 10),
                'grow_policy':trial.suggest_categorical ('grow_policy', ['Lossguide','SymmetricTree']),
                'max_bin': trial.suggest_int("max_bin", 20, 500),
                'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 1, 100),
                "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"])
            }
    
    if params['grow_policy'] == 'Lossguide':
        params['max_leaves']:trial.suggest_int('max_leaves', 1, 20)
    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1)
        
    
    kf = KFold(n_splits = K_FOLDS, shuffle=True,random_state = SEED)


    y_predicted_cat,models_cat,y_oof_cat,oof_score_cat =train_catbo(train,features,target,test,params,
                                                                    kf=kf, split_by = 'pm2_5')
    
        
    return cust_score(y_oof_cat).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

In [ ]:
# features
# study.best_params
# {'learning_rate': 0.09336356367527626,
#  'depth': 8,
#  'l2_leaf_reg': 0.00033354798569319656,
#  'random_strength': 10.395143072706773,
#  'grow_policy': 'SymmetricTree',
#  'max_bin': 111,
#  'min_data_in_leaf': 94,
#  'bootstrap_type': 'Bernoulli',
#  'subsample': 0.4351151265075187}### params with MAE, metric
# cat_b_params = {'learning_rate': 0.07574755563809993, 'depth': 6, 'l2_leaf_reg': 0.39323612155779863, 'random_strength': 0.20259568870497338, 'grow_policy': 'Lossguide', 'max_bin': 67, 'min_data_in_leaf': 10, 'bootstrap_type': 'Bernoulli', 'subsample': 0.9244602042319705}

In [ ]:
# study.direction
K_FOLDS = 5
# kf = KFold(n_splits=N_split)
SEED = 43
import optuna

def objective(trial):
   
    params = { 'objective': 'reg:squarederror',
              'eval_metric': 'mae',
#             'tree_method': 'hist',
            'grow_policy' : trial.suggest_categorical ('grow_policy', ['lossguide','depthwise']),
            'learning_rate':trial.suggest_loguniform('learning_rate', 0.01, 0.2),
            'max_depth': trial.suggest_int('max_depth', 3, 20),# a virer avec'depthwise'
#             'reg_alpha': trial.suggest_loguniform('reg_alpha', 1, 10),
#             'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-15, 10.0),
            'max_delta_step':trial.suggest_int('max_delta_step', 1, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
            'colsample_bytree':trial.suggest_loguniform('colsample_bytree', 0.4, 1.0),
            'subsample': trial.suggest_loguniform('subsample', 0.4, 1.0),
            'seed':1
                }
    if params['grow_policy'] == 'lossguide':
        params['max_leaves'] = trial.suggest_int('max_leaves', 1, 100)   
        
    kf = KFold(n_splits = K_FOLDS, shuffle=True,random_state = SEED)

    y_predicted_xgb,models_xgb,y_oof_xgb,oof_score_xgb=train_xgb(train,features,target,test,
                                                             params,kf,'pm2_5',num_iter=1000,es = 30,ve = 500)
        
    return cust_score(y_oof_xgb).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

In [ ]:
# 4 folds
# xgb_params = {'grow_policy': 'depthwise', 'learning_rate': 0.10769618165955801, 'max_depth': 14, 'max_delta_step': 2, 'min_child_weight': 132, 'colsample_bytree': 0.6353119293029151, 'subsample': 0.6254457606715289,'tree_method': 'hist'}
# xgb_params = {'grow_policy': 'lossguide', 'learning_rate': 0.03982304720613181, 'max_depth': 12, 'max_delta_step': 10, 'min_child_weight': 133, 'colsample_bytree': 0.8709673399493595, 'subsample': 0.7129355859703879, 'max_leaves': 34}

In [ ]:
# N_split = 2
# kf = KFold(n_splits=N_split)
# EARLY_STOPPING_ROUNDS = 30
N_ESTIMATORS = 5000
K_FOLDS = 3
import optuna

def objective(trial):
    params = {
            'objective': 'regression',
            'metric': 'mae',
            'n_estimators':5000,
            'verbose':-1,
            'learning_rate': trial.suggest_float("learning_rate", 0.04,0.4),
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'colsample_bytree':trial.suggest_float("colsample", 0.1,0.3),
            'subsample': trial.suggest_float("subsample", 0.1,0.3),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_child_samples': trial.suggest_int('min_child_samples', 3, 2000),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
            'cat_smooth':trial.suggest_int('cat_smooth', 1, 100)
            }
    
    kf = KFold(n_splits = K_FOLDS, shuffle=False,random_state = None)
    y_predicted_lgb,models_lgb,y_oof_lgb,oof_score_lgb,features_importance_lgb = train_lgb(train,features,
                                                                                  target,test,params = params,kf = kf,split_by = 'pm2_5') 
    return cust_score(y_oof_lgb).sort_values(by = 'err', ascending = False)['err'].iloc[:4].mean()


study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=20)
best_params_lgb = {'objective': 'regression',
            'metric': 'mae',
            'n_estimators':5000,
            'verbose':-1,
    'learning_rate': 0.0635938724410916,
 'num_leaves': 31,
 'colsample': 0.2556455958822984,
 'subsample': 0.29787449702328844,
 'max_depth': 10,
 'min_child_samples': 16,
 'reg_alpha': 1.4868037975722531e-06,
 'reg_lambda': 0.0005428092641970505,
 'cat_smooth': 95} 

In [ ]:
study.best_params